In [ ]:
# import R1 sequences
qiime tools import \
--type 'SampleData[SequencesWithQuality]' \
--input-path ../reads/R1 \
--input-format CasavaOneEightSingleLanePerSampleDirFmt \
--output-path demux-r1.qza

In [ ]:
qiime demux summarize \
--i-data demux-r1.qza \
--o-visualization demux-r1.qzv

In [ ]:
qiime dada2 denoise-single \
--i-demultiplexed-seqs demux-r1.qza \
--p-trunc-len 0 \
--p-max-ee 2 \
--p-trunc-q 2 \
--p-n-threads 2 \ 
--o-table ITS-r1-table.qza \
--o-representative-sequences ITS-r1-rep-seqs.qza \
--o-denoising-stats r1-denoising-stats.qza

In [ ]:
qiime feature-table tabulate-seqs \
--i-data ITS-r1-rep-seqs.qza \
--o-visualization ITS-r1-rep-seqs.qzv

In [ ]:
qiime feature-table summarize \
--i-table ITS-r1-table.qza \
--o-visualization ITS-r1-table.qzv

In [ ]:
# reindexing of sample names - from numbers to w#
qiime feature-table group \
  --i-table ITS-r1-table.qza \
  --p-axis sample \
  --m-metadata-file BNW-skin-metadata-index.tsv \
  --m-metadata-column SampleID-new \
  --p-mode sum \
  --o-grouped-table ITS-r1-reindexed-table.qza

In [ ]:
qiime feature-table summarize \
  --i-table ITS-r1-reindexed-table.qza \
  --m-sample-metadata-file BNW-skin-metadata.tsv \
  --o-visualization ITS-r1-reindexed-table.qzv

In [ ]:
# download the UNITE QIIME reference sequences and taxonomy files
wget https://files.plutof.ut.ee/public/orig/98/AE/98AE96C6593FC9C52D1C46B96C2D9064291F4DBA625EF189FEC1CCAFCF4A1691.gz

In [ ]:
# decompress file
tar xzf 98AE96C6593FC9C52D1C46B96C2D9064291F4DBA625EF189FEC1CCAFCF4A1691.gz

In [ ]:
# move into developer environment
cd sh_qiime_release_04.02.2020/developer/

In [ ]:
# fix formatting errors that prevent importation of the reference sequences. 
# There are white spaces and lower case letters that need to be converted
awk '/^>/ {print($0)}; /^[^>]/ {print(toupper($0))}' sh_refs_qiime_ver8_99_04.02.2020_dev.fasta | tr -d ' ' > sh_refs_qiime_ver8_99_04.02.2020_dev_uppercase.fasta

In [ ]:
# import UNITE reference sequences
qiime tools import \
--type 'FeatureData[Sequence]' \
--input-path sh_refs_qiime_ver8_99_04.02.2020_dev_uppercase.fasta \
--output-path unite-ver8-seqs_99.qza

In [ ]:
# import taxonomy file
qiime tools import \
--type 'FeatureData[Taxonomy]' \
--input-path sh_taxonomy_qiime_ver8_99_04.02.2020_dev.txt \
--output-path unite-ver8-taxonomy_99.qza \
--input-format HeaderlessTSVTaxonomyFormat

In [ ]:
# train the classifier
qiime feature-classifier fit-classifier-naive-bayes \
--i-reference-reads unite-ver8-seqs_99.qza \
--i-reference-taxonomy unite-ver8-taxonomy_99.qza \
--o-classifier unite-ver8-99-classifier.qza

In [ ]:
qiime feature-classifier classify-sklearn \
  --i-classifier sh_qiime_release_04.02.2020/developer/unite-ver8-99-classifier.qza \
  --i-reads ITS-r1-rep-seqs.qza \
  --o-classification taxonomy-single-end.qza

In [ ]:
qiime metadata tabulate \
--m-input-file taxonomy-single-end.qza \
--o-visualization taxonomy-single-end.qzv

In [ ]:
qiime taxa barplot \
--i-table ITS-r1-reindexed-table.qza \
--i-taxonomy taxonomy-single-end.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--o-visualization taxa-bar-plots-r1.qzv

In [ ]:
qiime diversity alpha-rarefaction \
--i-table ITS-r1-reindexed-table.qza \
--m-metadata-file BNW-skin-metadata.tsv   \
--o-visualization ITS-r1-reindexed-table-rarefaction.qzv \
--p-min-depth 500 \
--p-max-depth 50000

In [ ]:
qiime feature-table filter-samples \
--i-table ITS-r1-reindexed-table.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--p-min-frequency 9778 \
--o-filtered-table ITS-r1-table-atleast-9778-reads.qza

In [ ]:
qiime feature-table summarize \
--i-table ITS-r1-table-atleast-9778-reads.qza \
--o-visualization ITS-r1-table-atleast-9778-reads.qzv

# Taxa barplot using filtered reads

In [ ]:
qiime taxa barplot \
--i-table ITS-r1-table-atleast-9778-reads.qza \
--i-taxonomy taxonomy-single-end.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--o-visualization ITS-r1-table-atleast-9778-reads-BARPLOT-PER-SAMPLE.qzv

In [ ]:
qiime feature-table group \
--i-table ITS-r1-table-atleast-9778-reads.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--m-metadata-column infectionstatus \
--p-mode mean-ceiling \
--p-axis sample \
--o-grouped-table ITS-r1-table-atleast-9778-reads-INFECTION-STATUS.qza

In [ ]:
qiime taxa barplot \
--i-table ITS-r1-table-atleast-9778-reads-INFECTION-STATUS.qza \
--i-taxonomy taxonomy-single-end.qza \
--m-metadata-file BNW-skin-metadata-infection.tsv \
--o-visualization ITS-r1-table-atleast-9778-reads-INFECTION-STATUS-BARPLOT.qzv

In [ ]:
qiime feature-table group \
--i-table ITS-r1-table-atleast-9778-reads.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--m-metadata-column bodysite \
--p-mode mean-ceiling \
--p-axis sample \
--o-grouped-table ITS-r1-table-atleast-9778-reads-BODY-SITE.qza

In [ ]:
qiime taxa barplot \
--i-table ITS-r1-table-atleast-9778-reads-BODY-SITE.qza \
--i-taxonomy taxonomy-single-end.qza \
--m-metadata-file BNW-skin-metadata-BS.tsv \
--o-visualization ITS-r1-table-atleast-9778-reads-BODY-SITE-BARPLOT.qzv

In [ ]:
qiime feature-table group \
--i-table ITS-r1-table-atleast-9778-reads.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--m-metadata-column wombatid \
--p-mode mean-ceiling \
--p-axis sample \
--o-grouped-table ITS-r1-table-atleast-9778-reads-WOMBAT.qza

In [ ]:
qiime taxa barplot \
--i-table ITS-r1-table-atleast-9778-reads-WOMBAT.qza \
--i-taxonomy taxonomy-single-end.qza \
--m-metadata-file BNW-skin-metadata-WOM.tsv \
--o-visualization ITS-r1-table-atleast-9778-reads-WOMBAT-BARPLOT.qzv

# Alpha/beta diversity code

In [ ]:
# calculate alpha diversity using shannon's index
qiime diversity alpha \
  --i-table ITS-r1-reindexed-table.qza \
  --p-metric shannon \      
  --o-alpha-diversity ITS-r1-shannon.qza
# visualise
qiime diversity alpha-group-significance \
  --i-alpha-diversity ITS-r1-shannon.qza \
  --m-metadata-file BNW-skin-metadata.tsv \
  --o-visualization ITS-r1-shannon-group-significance.qzv

In [ ]:
# alpha diversity w. chao1
qiime diversity alpha \
  --i-table ITS-r1-reindexed-table.qza \
  --p-metric chao1 \      
  --o-alpha-diversity ITS-r1-chao1.qza
# visualise
qiime diversity alpha-group-significance \
  --i-alpha-diversity ITS-r1-chao1.qza \  
  --m-metadata-file BNW-skin-metadata.tsv  \
  --o-visualization ITS-r1-chao1-group-significance.qzv

In [ ]:
# alpha diversity w. Pielou's evenness
qiime diversity alpha \                   
  --i-table ITS-r1-table.qza \          
  --p-metric pielou_e \                                                      
  --o-alpha-diversity ITS-r1-pielou_e.qza
# visualise
qiime diversity alpha-group-significance \
  --i-alpha-diversity ITS-r1-pielou_e.qza \
  --m-metadata-file BNW-skin-metadata.tsv  \
  --o-visualization ITS-r1-pielou_e-group-significance.qzv

In [ ]:
# beta-diversity between body sites
qiime diversity beta \
--i-table ITS-r1-reindexed-table.qza \
--p-metric braycurtis \
--o-distance-matrix ITS-r1-braycurtis-unweighted_unifrac_distance_matrix.qza

qiime diversity beta-group-significance \
  --i-distance-matrix ITS-r1-braycurtisunweighted_unifrac_distance_matrix.qza \
  --m-metadata-file BNW-skin-metadata.tsv \
  --m-metadata-column bodysite \
  --o-visualization ITS-r1-braycurtis-unweighted-unifrac-body-site-significance.qzv \
  --p-pairwise

In [ ]:
# beta-diversity between wombats
qiime diversity beta-group-significance \
  --i-distance-matrix ITS-r1-braycurtisunweighted_unifrac_distance_matrix.qza \
  --m-metadata-file BNW-skin-metadata.tsv \
  --m-metadata-column wombatid \
  --o-visualization ITS-r1-braycurtis-unweighted-unifrac-wombatid-significance.qzv \
  --p-pairwise

In [ ]:
# beta-diversity between infection status
qiime diversity beta-group-significance \
  --i-distance-matrix ITS-r1-braycurtisunweighted_unifrac_distance_matrix.qza \
  --m-metadata-file BNW-skin-metadata.tsv \
  --m-metadata-column infectionstatus \
  --o-visualization ITS-r1-braycurtis-unweighted-unifrac-infection-status-significance.qzv \
  --p-pairwise

# Export files to analyse in R